<div style="direction:rtl;line-height:300%;">
<font face="XB Zar" size=5>
<div align=center>
<font face="B Titr" size=5>
<p></p><p></p>
بسمه تعالی
<p></p>
</font>
<p></p>
<font>
<br>
درس بازیابی پیشرفته اطلاعات
<br>
مدرس: دکتر بیگی
</font>
<p></p>
<br>
<font>
<b>تمرین سوم</b>
</font>
<br>
<br>
موعد تحویل: ۴ دی <br>
<br>
<br>
<br>
<font>
دانشگاه صنعتی شریف
<br>
دانشکده مهندسی کامپیوتر
<br>
<br>
</font>
</div>
</font>
</div>

<span style="color: red;">
<div dir='rtl'>

<b> نام و نام‌‌خانوادگی: </b>
<!-- YOUR NAME HERE -->

<b> شماره دانشجویی: </b>
<!-- YOUR STUDENT ID HERE -->

<b> لینک colab: </b>

<!-- UPLOAD YOUR NOTEBOOK TO GOOGLE COLAB AND MAKE SURE TO RUN ALL OF ITS CELLS -->
</div>
</span>

<div dir='rtl'>
<h1> 
مقدمه
</h1>
<p>
در این تمرین قصد داریم به مباحث زیر بپردازیم:
<li> embedding </li>
<li> کلاسه‌بندی متن </li>
<li> کاهش ابعاد </li>
<li> خوشه‌بندی متن و هرس‌کردن خوشه‌ها</li>

دیتاست این تمرین از دیتاست‌های kaggle انتخاب شده‌است و لینک آن در بخش اول تمرین در اختیار شما قرار داده شده است.

کتابخانه‌های مورد نظرتان را هم می‌توانید در اولین سل نوت‌بوک فراخوانی کنید. 
</p>
</div>

In [74]:
import opendatasets as od
import pandas as pd
import numpy as np
import string
import nltk
import torch
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.metrics import edit_distance
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import fasttext
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/mhdi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<div dir='rtl'>
<h1>1.
دریافت و آماده‌سازی دیتاست
</h1>
<p>
دیتاست استفاده شده در این تمرین، مجموعه‌ای عناوین، خلاصه، و ژانر چندین کتاب است. 
این دیتاست در kaggle موجود است. 
ابتدا این دیتاست را با استفاده از kaggle api دریافت کنید و سپس آن را لود کنید.
<br/>
لینک دیتاست: https://www.kaggle.com/datasets/athu1105/book-genre-prediction
<br>
<i>در صورتی که با خطای 443 مواجه شدید، یا از پراکسی استفاده کنید یا از کولب.</i>
</p>
</div>

In [3]:
df = pd.read_csv('./data/phase-3/data.csv')

In [4]:
df.head()

,index,title,genre,summary
0,0,Drowned Wednesday,fantasy,Drowned Wednesday is the first Trustee among ...
1,1,The Lost Hero,fantasy,"As the book opens, Jason awakens on a school ..."
2,2,The Eyes of the Overworld,fantasy,Cugel is easily persuaded by the merchant Fia...
3,3,Magic's Promise,fantasy,The book opens with Herald-Mage Vanyel return...
4,4,Taran Wanderer,fantasy,Taran and Gurgi have returned to Caer Dallben...


<div dir='rtl'>
<p>
حال در این مرحله، به پیش‌پردازش متن می‌پردازیم. این پیش‌پردازش باید روی هر دو ستون title و summary اعمال شود.
برای پیش‌پردازش نیازی نیست که هرکدام از اعمال پیش‌پردازش را خودتان مانند تمارین قبل پیاده کنید. برای پیاده‌سازی تابع زیر می‌توانید از کتابخانه‌های معمول برای این کار بهره ببرید.
</p>
</div>

In [5]:
stopwords_domain = stopwords.words('english')

In [6]:
def preprocess_text(text, minimum_length=1, stopword_removal=True, stopwords_domain=stopwords_domain, lower_case=True,
                       punctuation_removal=True):
    """
    preprocess text by removing stopwords, punctuations, and converting to lowercase, and also filter based on a min length
    for stopwords use nltk.corpus.stopwords.words('english')
    for punctuations use string.punctuation

    Parameters
    ----------
    text: str
        text to be preprocessed
    minimum_length: int
        minimum length of the token
    stopword_removal: bool
        whether to remove stopwords
    stopwords_domain: list
        list of stopwords to be removed base on domain
    lower_case: bool
        whether to convert to lowercase
    punctuation_removal: bool
        whether to remove punctuations
    """

    normalized_tokens = word_tokenize(text)

    if stopword_removal:
        stopwords_domain = [x.lower() for x in stopwords_domain]
        normalized_tokens = [word for word in normalized_tokens if word.lower() not in stopwords_domain]

    if punctuation_removal:
        normalized_tokens = [word for word in normalized_tokens if word not in string.punctuation]

    if lower_case:
        normalized_tokens = [word.lower() for word in normalized_tokens if len(word) > minimum_length]

    else:
        normalized_tokens = [word for word in normalized_tokens if len(word) > minimum_length]

    return normalized_tokens



In [7]:
df["summary_preprocessed"] = df["summary"].apply(lambda x: preprocess_text(x))
df["title_preprocessed"] = df["title"].apply(lambda x: preprocess_text(x))

<div dir="rtl">
در نهایت بعد از اپلای کردن پیش‌پردازش روی هر دو ستون، دو ستون پیش‌پردازش شده را با هم ادغام کنید و آن را در یک آرایه به نام X قرار دهید.
</div>

In [8]:
X = df['title_preprocessed'] + df['summary_preprocessed']
X_text = [' '.join(x) for x in X]

In [9]:
X_text[1]

"lost hero book opens jason awakens school bus unable remember anything past sitting next piper mclean leo valdez call name say girlfriend best friend respectively three part class field trip grand canyon arrive classmate dylan turns venti storm spirit attacks trio trip leader coach gleeson hedge ensuing fight jason surprises everyone including one coins turns sword uses battle storm spirits coach hedge reveals satyr fight taken captive fleeing spirit battle flying chariot arrives rescue trio one people annabeth upset discovers missing boyfriend percy jackson expected annabeth seeking percy told vision goddess hera look `` guy one shoe '' turns jason shoe destroyed fight jason piper leo told demigods taken back camp half-blood meet greek demigod children like leo revealed son hephaestus piper daughter aphrodite jason son zeus though hera tells champion jason later discovers full brother zeus 's demigod daughter thalia grace hunter artemis shortly arrive three given quest rescue hera ca

<div dir="rtl">
<h2>1-1.
Embedding
</h2>

در این قسمت از fasttext کمک می‌گیریم تا به یک embedding اولیه برای هر کتاب برسیم.
    با استفاده از داده‌هایی که داریم یک مدل fasttext آموزش دهید که برای هر توکن یک امبدینگ ۱۰۰تایی بدهد.
    در مرحله‌ی بعد میانگین وزن دار امبدینگ های fasttext
        توکن‌های ورودی (چکیده + عنوان)
    را بر اساس tfidif آن‌ها محاسبه کنید و به امبدینگ نهایی متن برسید.
    <br>
    در واقع به عبارت ساده‌تر بر اساس میانگین وزن‌دار که وزن‌های ما tfidf توکن‌ها می‌باشد به امبدینگ نهایی متن بر اساس fasttext می‌رسیم.
</div>

In [43]:
class FastText:

    def __init__(self, preprocessor=None, method='skipgram'):
        self.method = method
        self.model = None
        self.preprocessor = preprocessor

    def train(self, texts):
        """
        train the fasttext model and save it into self.model
        Parameters
        ----------
        texts: list of list of str
        """
        file_name = './data/phase-3/data_cleaned.txt'
        with open(file_name, 'w') as f:
            for item in texts:
                f.write("%s\n" % item)
            f.close()

        self.model = fasttext.train_unsupervised(file_name, model=self.method)

    def get_query_embedding(self, query, tf_idf_vectorizer):
        """
        get the embedding of the query. You can use the tf_idf_vectorizer to get the weights of the words in the query. preprocess the query using self.preprocessor if it is not None
        Parameters
        ----------
        query: str
        tf_idf_vectorizer: TfidfVectorizer
        Returns embedding of the query
        """
        embedded_query = np.zeros(100)
        query = self.preprocessor(query)
        tf_idf_scores = tf_idf_vectorizer.transform([' '.join(query)]).toarray()
        for word in query:
            if word in tf_idf_vectorizer.vocabulary_.keys():
                word_indice = tf_idf_vectorizer.vocabulary_[word]
                embedded_query += tf_idf_scores[0][word_indice] * self.model.get_word_vector(word)

        return embedded_query

    def save_FastText_model(self, path='./models/FastText_model.bin'):
        self.model.save_model(path)

    def load_FastText_model(self, path="./models/FastText_model.bin"):
        self.model = fasttext.load_model(path)

    def prepare(self, dataset, mode, save=False):
        if mode == 'train':
            self.train(dataset)
        if mode == 'load':
            self.load_FastText_model()
        if save:
            self.save_FastText_model()

In [44]:
FastText_model = FastText(preprocessor=preprocess_text)
FastText_model.prepare(X_text, mode='train', save=True)
FastText_model.prepare(X, mode='load')

Read 0M words
Number of words:  20282
Number of labels: 0
Progress: 100.0% words/sec/thread:   65583 lr:  0.000002 avg.loss:  2.271513 ETA:   0h 0m 0s100.1% words/sec/thread:   64968 lr: -0.000030 avg.loss:  2.271576 ETA:   0h 0m 0sWarning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.
Progress: 100.0% words/sec/thread:   64967 lr:  0.000000 avg.loss:  2.271576 ETA:   0h 0m 0s


In [45]:
word = "society"
FastText_model.model.get_nearest_neighbors(word)

[(0.880240261554718, 'societies'),
 (0.8100507259368896, 'agriculture'),
 (0.8066205978393555, 'equality'),
 (0.8012783527374268, 'lifestyle'),
 (0.7936832308769226, 'sociology'),
 (0.7921492457389832, 'agricultural'),
 (0.7878493666648865, 'fertility'),
 (0.786555290222168, 'variety'),
 (0.782918393611908, 'equals'),
 (0.7827007174491882, 'popularity')]

<div dir="rtl">
در این مرحله آرایه X را روی TFIDF فیت می‌کنیم.
</div>

In [46]:
class TF_IDF:

    def __init__(self):
        self.vectorizer = TfidfVectorizer()

    def fit_vectorizer(self, data):
        """
        fit the vectorizer on the data
        Parameters
        ----------
        data: list of list of str
        """
        self.vectorizer.fit_transform(data)

In [47]:
TF_IDF_model = TF_IDF()
TF_IDF_model.fit_vectorizer(X_text)

In [49]:
text = "society of the snow"
embedded_text = FastText_model.get_query_embedding(text, TF_IDF_model.vectorizer)
print(embedded_text)

[ 0.07685816  0.15103993 -0.31325506 -0.31091463 -0.11035508 -0.06025137
 -0.33890378 -0.20461031  0.19143126 -0.01584841  0.10076773  0.30836532
  0.07478167  0.04833817  0.35057064  0.16702574 -0.2122911  -0.23538411
 -0.17856634 -0.29422837 -0.13177812  0.2476757   0.44564785 -0.65641227
 -0.02192403 -0.2110163   0.57409605  0.04755806 -0.08134577  0.25422455
  0.12808537  0.20377593 -0.74035645 -0.00785394  0.52432304 -0.30830564
 -0.12534711 -0.33635902 -0.08596513  0.13402439 -0.14908559 -0.17038143
 -0.40552511 -0.16403412 -0.03879646 -0.03073727  0.3090703   0.02828454
  0.05911007  0.08717421 -0.53820047 -0.01005109 -0.55456436  0.06239751
 -0.20141677 -0.41153832 -0.03252132 -0.20446468 -0.29354015  0.53638878
  0.37253731  0.15439685 -0.55014616  0.28978732 -0.40171458  0.21939173
 -0.26290529 -0.42341182  0.036148   -0.47445785  0.05072556  0.26519767
  0.21300658  0.0514566  -0.02343663  0.35361504 -0.31836375 -0.46709597
 -0.19775754  0.22024759  0.10076064  0.06863415 -0

<div dir="rtl">
در نهایت، تمامی entry های درون X را به صورت امبدینگ دربیاورید و آن را در X ذخیره کنید.
</div>

In [50]:
embeddings = []
for text in tqdm(X_text):
    embeddings.append(FastText_model.get_query_embedding(text, TF_IDF_model.vectorizer))

  0%|          | 0/4657 [00:00<?, ?it/s]

100%|██████████| 4657/4657 [00:09<00:00, 473.31it/s]


<div dir='rtl'>
<h2>1-2. 
آماده‌سازی داده تمرین و تست
</h2>
<p>
در این بخش ابتدا می‌خواهیم تا داده‌ای که می‌خواهیم بر اساس آن کلاسه‌بندی کتاب‌ها را انجام دهیم، که همان ژانر کتاب‌ها است را انکود کنیم.
<br>
سپس با جداسازی داده آموزش و تست، به آموزش مدل‌های کلاسه‌بند می‌پردازیم.
</p>
</div>

In [60]:
labels = df['genre'].unique()
label_to_id = {}
id_to_label = {}
for i, label in enumerate(labels):
    label_to_id[label] = i
    id_to_label[i] = label

labels = df['genre'].apply(lambda x: label_to_id[x])

In [66]:
x_train, x_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2)

<div dir="rtl">
<h1>2. 
کلاسه‌بندی متن
</h1>
در این بخش می‌خواهیم تا با استفاده از داده‌هایی که داریم، سه مدل کلاسه‌بندی متن را آموزش دهیم و عملکرد آن‌ها را با هم مقایسه کنیم.
</div>

<div dir="rtl">
<h2>2-1. 
SVM و Naive Bayes
</h2>
با استفاده از توابع آماده در sklearn، کلاسه‌بند SVM و Naive Bayes را روی داده آموزش تمرین دهید. سپس، ژانر داده تست را با استفاده از مدل آموزش داده شده پیش‌بینی کنید و آن را در آرایه‌های NB_prediction و SVM_prediction ذخیره کنید.</div>

In [75]:
NB = GaussianNB()
NB.fit(x_train, y_train)

SVM = svm.SVC()
SVM.fit(x_train, y_train)


SVC()

In [76]:
NB_prediction = NB.predict(x_test)
SVM_prediction = SVM.predict(x_test)

<div dir="rtl">
<h2>2-2.  
Rocchio
</h2>
می‌خواهیم روش Rocchio را از
پایه پیاده‌سازی کنیم.
با توجه به شواهد نوشته شده هر تابع را کامل کنید و با آموزش مدل روی داده‌های train
لیبل‌های داده‌های test
را پیش بینی کنید و آن را در rocchio_prediction ذخیره کنید.
</div>

In [26]:
class RocchioTextClassifier:
    def __init__(self, preprocessor=None):
        self.preprocessor = preprocessor
        self.centroids = None

    def calculate_centroids(self, X, y):
        """
        Calculates the centroids of each class in the dataset. A centroid is defined as the mean vector of all the feature vectors in a class.

        Parameters
        ----------
        X : np.ndarray
            An array contaning the feature vectors of each sentence.

        y : np.ndarray
            An array containing the class labels for each feature vector in X.

        Sets self.centeroids as a dictionary where keys are unique class labels from 'y', and values are the calculated centroids (mean vectors) for each class.
        """
        self.centroids = {}
        # TODO
        pass

    def predict_label(self, x):
        """
        Classifies a new instance by finding the class whose centroid is closest to the new instance's vector.

        Parameters
        ----------
        X : np.ndarray
            An array of a new instance to be classified.

        Returns
        -------
        object
            The predicted label for the input.
        """
        # TODO
        pass
        return label

    def predict(self, X):
        """
        Classifies the array X

        Parameters
        ----------
        X : np.ndarray
            An array of new instances to be classified.

        Returns
        -------
        object
            The predicted label for each input.
        """
        return np.array([self.predict_label(x) for x in X])


In [27]:
# TODO: Train your Rocchio implementation on X_train and Y_train

In [28]:
# TODO: Predict on X_test using the trained model

<div dir="rtl">
<h2>2-3. 
ارزیابی
</h2>
برای هر سه روش پیاده‌سازی شده، معیار‌های ارزیابی زیر را بررسی کنید.
<br>
f1 score, accuracy, precision, recall
<br>
سپس، نتایج به دست آمده را با هم در چهار نمودار مقایسه کنید.
</div>

In [29]:
# TODO: Run evaluation metrics on the predictions

In [30]:
# TODO: Plot evaulations

In [31]:
# TODO: Draw confusion matrix

<div dir="rtl">
کدام مدل از همه دقیق‌تر عمل کرد؟ نتیجه‌گیری و تحلیل خود از نتایج ارزیابی را گزارش دهید.
</div>

<span style="color: red;">#TODO: Write your answer in here.</span>


<div dir="rtl">
<h1>3. 
کاهش ابعاد و خوشه‌بندی متن
</h1>
</div>

<div dir="rtl">
<h2>3-1. 
کاهش ابعاد
</h2>
</div>

<div dir="rtl">
<h3>3-1-1. 
PCA
</h3>
یکی از روش‌های کاهش ابعاد، PCA است. با استفاده از پیاده‌سازی آن در کتابخانه sklearn، ابعاد ویژگی‌های X را کاهش دهید.
<br>
سپس با استفاده از explained_variance_ratio_ در الگوریتم PCA  نشان دهید که با وجود یک ترشولد 90 درصد تا چه میزان میتوان ابعاد ویژگی ها را کم تر کرد.
</div>

In [32]:
def reduce_dimension(embedding, n_components):
    """
    Performs dimensional reduction using PCA with n components left behind

    Parameters
    ----------
    embeddings : List
        A list of embeddings of documents

    n_components: int
        Number of components to keep

    Returns a list of reduced embeddings
    """
    # TODO
    pass

<div dir="rtl">
در این قسمت می‌توانید برای شهود بهتر، نمودار رسم کنید و همچنین برای ساده‌تر شدن کار، از pipeline‌های sklearn بهره ببرید. 
<br>
<i> انجام این کار‌ها صرفا توصیه است و اجباری نیست. </i>
</div>

In [33]:
# TODO: Findout the most reduced dimension which has 90% cutoff explained variance.

<div dir="rtl">
<h3>3-1-2. 
TSNE
</h3>
<br>
     برای اینکه بتوانید در مراحل بعدی، نتایج خوشه‌بندی را مشاهده کنید، در این قسمت به پیاده‌سازی تابع کاهش بعد بردارهای جاسازی با استفاده از روش T-SNE می‌پردازید.
برای اینکار تابع convert_to_2d_tsne را پیاده‌سازی می‌کنید که لیستی از بردارهای جاسازی را به عنوان ورودی دریافت می‌کند و در خروجی، لیستی از بردارهای جاسازی کاهش بعد داده شده به دو بعد را تولید می‌کند. برای پیاده سازی این تابع می‌توانید از کتابخانه‌های آماده استفاده کنید.
<br>
توجه کنید که از بردارهای خروجی این قسمت <u>صرفا برای رسم نمودار</u> استفاده می‌کنید و تمامی مراحلی که در ادامه طی می‌کنید (به جز رسم نمودار)، باید با استفاده از بردارهای کاهش بعد داده <u>نشده</u> انجام شوند.
</div>


In [34]:
from typing import List
from sklearn.manifold import TSNE
def convert_to_2d_tsne(emb_vecs):
    """
    Converts each raw embedding vector to 2d vector

    Parameters
    ----------
    emb_vecs : List
        A list of vectors

    Returns a list of 2d vectors
    """

    # TODO: Convert each input vector to 2d vector
    pass

In [35]:
# TODO: Uncomment the following code
# X_2d = convert_to_2d_tsne(np.array(X))

<div dir="rtl">
<h2>3-2. 
خوشه‌بندی
</h2>
</div>

<div dir="rtl">
<h3>3-2-1. 
K-Means
</h3>

<div dir="rtl">
در این قسمت، شما ابتدا الگوریتم خوشه‌بندی K-means را 
<u><b>از پایه</b></u>
 پیاده‌سازی می‌کنید.

</div>

In [36]:
from typing import List
import random

def cluster_kmeans(emb_vecs, n_clusters):
    """
    Clusters input vectors using K-means method

    Parameters
    ----------
    emb_vecs : List
        A list of vectors

    n_clusters: int
        Number of clusters

    Returns
    -------
    list
        Two lists: 1) A list containing cluster centers 2) A list containing cluster index for each input vector
    """

    # TODO: Implement K-means method from scratch
    # You can't use sklearn.cluster.KMeans for clustering
    # implement kmeans clustering here
    emb_matrix = None

    # Randomly initialize centroids
    initial_centroids = None

    for _ in range(None):  # You can adjust the number of iterations
        # Compute distances between data points and centroids
        distances = None

        # Assign each point to the closest centroid
        cluster_indices = None

        # Update centroids
        new_centroids = []


    return initial_centroids.tolist(), cluster_indices.tolist()

<div dir="rtl">
 با استفاده از K-Means خوشه‌های اسناد را ایجاد کنید. الگوریتم را با استفاده از چند مقدار مختلف تعداد خوشه‌ها (k) اجرا کنید. در هربار اجرا، با استفاده از تعدادی از اسناد موجود در هر خوشه، موضوع آن خوشه را تعیین کرده و خوشه‌بندی حاصله را با استفاده از بردار‌های دوبعدی قسمت قبل، رسم کنید. با اینکار، پیاده‌سازی خود و همچنین کارایی این الگوریتم در خوشه‌بندی اسناد و قرار دادن اسناد مشابه در خوشه‌های یکسان را بررسی کنید.
<br>
 نمودار silhouette score برای مقدار‌های مختلف k را رسم کرده و silhouette analysis برای انتخاب k مناسب انجام دهید. 
 همچنین با استفاده از داده‌های دارای برچسب، مقدار purity به ازای k را رسم کرده و مقدار purity برای k نهایی را گزارش کنید.
<br>
</div>

In [37]:

def purity_score(y_true, y_pred):
    """
    Calculate the purity score for the given cluster assignments and ground truth classes

    y_true: list
        ground truth labels for each document

    y_pred: list
        predicted labels for each document

    Returns a purity score between 0.0 and 1.0 (higher is better)
    """
    #TODO
    pass

In [38]:
# TODO: Calculate silhouette score and purity score

In [39]:
# TODO: Plot silhouette score for different value of k (at least 5 different k values)

In [40]:
#TODO: plot purity for different value of k (at least 5 different k values)

<div dir="rtl">
با استفاده از نمودارهای رسم شده توضیح دهید بهترین k برای انتخاب در داده ما با استفاده از الگوریتم K-Means چیست؟
چرا؟
</div>

<span style="color: red;">#TODO: Write your answer in here.</span>

<div dir="rtl">
<h3>3-2-2. 
Hierarchical clustering
</h3>
</div>

<div dir="rtl">
تکنیک خوشه‌بندی سلسله مراتبی یکی از تکنیک‌های خوشه‌بندی در یادگیری ماشین است. در این قسمت شما می‌توانید از لایببری scipy یا هر لایبرری دیگری در پایتون استفاده کنید تا داده‌ها را به صورت سلسله‌مراتبی خوشه‌بندی کنید. سپس می‌توانید خوشه‌ها را با matplotlib مشاهده کنید.
</div>

In [41]:
# TODO: Perform hierarchical clustering on X

In [42]:
# TODO: Plot dendrogram